# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv ../src/.env


In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")

'''# I have the first 50 of SP-500 data tickers and ran download and featurize on it. 
This is to show the dask time saving capabilities, thats why the directory structure below has changed, yu may need to adjust.'''

parquet_files = glob(os.path.join(PRICE_DATA, '***/*.parquet/*.parquet')) 
parquet_files

dd_px = dd.read_parquet(parquet_files, calculate_divisions=True, index = 'ticker') # calculating divisions so we can do rolling returns later on
dd_px.columns


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sector',
       'subsector', 'year'],
      dtype='object')

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
dd_px = dd_px.rename(columns={"Adj Close":"Adj_Close"}) # The famous "Adj Close" had a space, so we rename that. 
dd_px.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume', 'sector',
       'subsector', 'year'],
      dtype='object')

In [5]:
# Write your code below.
dd_fet = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1), Adj_Close_lag_1 = x['Adj_Close'].shift(1))
).assign(
    returns = lambda x: x['Adj_Close']/x['Adj_Close_lag_1'] - 1
).assign(
    hi_lo_range = lambda x: (x['High'] - x['Low'])
))

C:\Users\Ramin\AppData\Local\Temp\ipykernel_13768\1520099133.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_fet = (dd_px.groupby('ticker', group_keys=False).apply(


In [6]:
dd_fet.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume', 'sector',
       'subsector', 'year', 'Close_lag_1', 'Adj_Close_lag_1', 'returns',
       'hi_lo_range'],
      dtype='object')

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [8]:
# Write your code below.
import time
df_fet = dd_fet.compute() # getting the pandas dataframe
df_fet = df_fet.reset_index()
start_pd = time.time()
df_fet = df_fet.groupby('ticker', group_keys=False).apply(
                   lambda x: x.assign(rolling_10_mean_return=df_fet['returns'].rolling(10).mean())
               )
stop_pd = time.time()
df_fet.head(20) # just to make sure the rolling mean is happening

C:\Users\Ramin\AppData\Local\Temp\ipykernel_13768\1333670442.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_fet = df_fet.groupby('ticker', group_keys=False).apply(


,ticker,Date,Open,High,Low,Close,Adj_Close,Volume,sector,subsector,year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,rolling_10_mean_return
0,A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.683865,4674353,Health Care,Life Sciences Tools & Services,2000,NaN,NaN,NaN,8.270744,NaN
1,A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.346909,4765083,Health Care,Life Sciences Tools & Services,2000,51.502148,43.683865,-0.076389,2.950645,NaN
2,A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.844173,5758642,Health Care,Life Sciences Tools & Services,2000,47.567955,40.346909,-0.062030,4.425964,NaN
3,A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.403214,2534434,Health Care,Life Sciences Tools & Services,2000,44.617310,37.844173,-0.038076,2.771820,NaN
4,A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.436821,2819626,Health Care,Life Sciences Tools & Services,2000,42.918453,36.403214,0.083334,4.962444,NaN
5,A,2000-01-10,49.356224,49.803291,48.327969,49.311516,41.825775,2148446,Health Care,Life Sciences Tools & Services,2000,46.494991,39.436821,0.060577,1.475323,NaN
6,A,2000-01-11,49.311516,49.311516,47.523247,48.640915,41.256969,1855985,Health Care,Life Sciences Tools & Services,2000,49.311516,41.825775,-0.013599,1.788269,NaN
7,A,2000-01-12,48.640915,48.640915,45.824390,47.657368,40.422729,1429874,Health Care,Life Sciences Tools & Services,2000,48.640915,41.256969,-0.020221,2.816525,NaN
8,A,2000-01-13,48.909157,49.937412,47.210300,48.372677,41.029469,1134337,Health Care,Life Sciences Tools & Services,2000,47.657368,40.422729,0.015010,2.727112,NaN
9,A,2000-01-14,47.925610,49.624462,47.925610,48.909157,41.484505,1316916,Health Care,Life Sciences Tools & Services,2000,48.372677,41.029469,0.011090,1.698853,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

No, we could have done that in dask. Yes, it would have been better to do that in Dask, because the same operation in pndas takes twice as long. You can see the timing below.

In [9]:
start_dd = time.time()
dd_fet['rolling_10_mean_return'] = dd_fet['returns'].rolling(10).mean() # we do not gorupby as we have divisions in the dask df setup already. 
stop_dd = time.time()
dd_fet.head(20) # just to make sure the rolling mean is happening

,Date,Open,High,Low,Close,Adj_Close,Volume,sector,subsector,year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,rolling_10_mean_return
ticker,,,,,,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.683865,4674353,Health Care,Life Sciences Tools & Services,2000,NaN,NaN,NaN,8.270744,NaN
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.346909,4765083,Health Care,Life Sciences Tools & Services,2000,51.502148,43.683865,-0.076389,2.950645,NaN
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.844173,5758642,Health Care,Life Sciences Tools & Services,2000,47.567955,40.346909,-0.062030,4.425964,NaN
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.403214,2534434,Health Care,Life Sciences Tools & Services,2000,44.617310,37.844173,-0.038076,2.771820,NaN
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.436821,2819626,Health Care,Life Sciences Tools & Services,2000,42.918453,36.403214,0.083334,4.962444,NaN
A,2000-01-10,49.356224,49.803291,48.327969,49.311516,41.825775,2148446,Health Care,Life Sciences Tools & Services,2000,46.494991,39.436821,0.060577,1.475323,NaN
A,2000-01-11,49.311516,49.311516,47.523247,48.640915,41.256969,1855985,Health Care,Life Sciences Tools & Services,2000,49.311516,41.825775,-0.013599,1.788269,NaN
A,2000-01-12,48.640915,48.640915,45.824390,47.657368,40.422729,1429874,Health Care,Life Sciences Tools & Services,2000,48.640915,41.256969,-0.020221,2.816525,NaN
A,2000-01-13,48.909157,49.937412,47.210300,48.372677,41.029469,1134337,Health Care,Life Sciences Tools & Services,2000,47.657368,40.422729,0.015010,2.727112,NaN


In [14]:
print(f'Calculating rolling average of returns for dd_fet (parquet) took {stop_dd - start_dd} seconds.')

Calculating rolling average of returns for dd_fet (parquet) took 0.013226747512817383 seconds.


In [15]:
print(f'Calculating rolling average of returns for dd_fet (Pandas) took {stop_pd - start_pd} seconds.')

Calculating rolling average of returns for dd_fet (Pandas) took 0.7940969467163086 seconds.
